In [ ]:
import cv2
import mediapipe as mp
import numpy as np
def make_white_transparent(image):
    if image.shape[2] == 4:
        b, g, r, a = cv2.split(image)
        mask = (b == 255) & (g == 255) & (r == 255)
        a[mask] = 0
        image = cv2.merge([b, g, r, a])
    else:
        b, g, r = cv2.split(image)
        a = np.ones(b.shape, dtype=b.dtype) * 255
        mask = (b == 255) & (g == 255) & (r == 255)
        a[mask] = 0
        image = cv2.merge([b, g, r, a])
    return image

def overlay(image, x, y, overlay_image, scale=1.0):
    #오버레이 이미지 크기 조정
    overlay_image_resized = cv2.resize(overlay_image, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)

    if overlay_image_resized.shape[2] == 4: 
        alpha = overlay_image_resized[:, :, 3] / 255.0
        overlay_image_rgb = overlay_image_resized[:, :, :3]
    else:
        alpha = np.ones(overlay_image_resized.shape[:2], dtype=overlay_image_resized.dtype)
        overlay_image_rgb = overlay_image_resized

    h, w = overlay_image_rgb.shape[:2]
    
    #오버레이 이미지를 붙일 영역의 좌표 계산
    start_x = max(x - w // 2, 0)
    start_y = max(y - h // 2, 0)
    end_x = min(start_x + w, image.shape[1])
    end_y = min(start_y + h, image.shape[0])

    #오버레이 이미지를 적절한 크기로 자르고, 알파 채널을 기반으로 오버레이
    overlay_area = image[start_y:end_y, start_x:end_x]

    overlay_image_cropped = overlay_image_rgb[:end_y - start_y, :end_x - start_x]
    alpha_cropped = alpha[:end_y - start_y, :end_x - start_x]

    for c in range(3):  #R, G, B 
        overlay_area[:, :, c] = (overlay_image_cropped[:, :, c] * alpha_cropped) + \
                               (overlay_area[:, :, c] * (1 - alpha_cropped))

mp_face_detection = mp.solutions.face_detection  
mp_drawing = mp.solutions.drawing_utils  
cap = cv2.VideoCapture(0)  

image_right_eye = make_white_transparent(cv2.imread('right_ear.png', cv2.IMREAD_UNCHANGED))  #200 x 200
image_left_eye = make_white_transparent(cv2.imread('left_ear.png', cv2.IMREAD_UNCHANGED))  #200 x 200
image_nose = make_white_transparent(cv2.imread('nose.png', cv2.IMREAD_UNCHANGED))  #200 x 200

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.75) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break
        
        image.flags.writeable = False
        # RGB로 변환
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        #이미지에 얼굴 감지 주석 그리기
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            for detection in results.detections:
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose_tip = keypoints[2]
                mouth_center = keypoints[3]
                right_ear_tragion = keypoints[4]
                left_ear_tragion = keypoints[5]

                #이미지의 절대 좌표로 변환
                h, w, _ = image.shape
                right_eye = (int(right_eye.x * w) + 10, int(right_eye.y * h) - 40)
                left_eye = (int(left_eye.x * w) + 10, int(left_eye.y * h) - 40)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                #각 특징점에 오버레이 이미지 붙이기
                overlay(image, *left_eye, image_right_eye, scale=0.29)  
                overlay(image, *right_eye, image_left_eye, scale=0.29)  
                overlay(image, *nose_tip, image_nose, scale=0.4) 

        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.7, fy=0.7))
        if cv2.waitKey(1) == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
